## Objetos = datos + codigo

Quiza una de las ideas mas dañinas para el diseño orientado objetos, es la definicion de un objeto como una estructura de datos con funciones. Aunque este sea el caso a nivel de implementacion, esta concepcion de un objeto limita mucho nuestra forma de pensar y por lo tanto de diseñar un sistema orientado a objetos. Una forma mas util es pensar en un objeto como una cosa que recibe mensajes. Despues de todo esa era la idea de Alan Kay cuando creo el termino programacion orientada a objetos. Por ejemplo, en la siguiente declaracion en C#:

In [16]:
int numero = 42;

numero.ToString(); 

La gente tiende a pensar que __numero__ es un dato. Sin embargo en la proxima linea vemos que recibe el mensaje ToString! __numero__ es un objeto! un objeto que representa un dato, pero un objeto al final de cuentas. Por otro lado, algo como:

In [17]:
Delegate range = Enumerable.Range;

range.DynamicInvoke(1, 10);

...es un objeto que representa codigo. Sin embargo la mayoria de la gente usa el concepto de puntero como una forma de explicar delegates en C#. Por que? Por que para ellos un objeto = datos + codigo. Cualquiera cosa fuera de esa definicion no es un objeto para ellos. Por cierto, asi es como se ve un puntero en C#:

In [18]:
unsafe
{
    int number = 42;
	int* p = &number;
}

Como puedes ver, la idea de datos y codigo queda abstraida bajo un solo concepto: objeto.

Igual de importante es que la idea del _flujo de control_ queda abstraida bajo el concepto de _mensaje_. Esto significa que cuando diseñamos un modelo de objetos, no pensamos en el flujo de datos entre modulos y funciones, sino en un flujo de mensajes entre diferentes objetos. Observa el siguiente diagrama:

In [19]:
sequenceDiagram
    Cajero->>Banco: Abrir cuenta
    Banco->>Cajero: Cuenta abierta
    Cajero->>Mensajero: Notificar cuenta abierta
    Mensajero->>Cajero: Notificacion enviada 

Pon atencion en los mensajes y el orden. Esa es la estructura del programa. Los objetos son reemplazables. 

In [20]:
sequenceDiagram
    ServicioCliente->>EmpresaTelefonica: Abrir cuenta
    EmpresaTelefonica->>ServicioCliente: Cuenta abierta
    ServicioCliente->>ServidorEmail: Notificar cuenta abierta
    ServidorEmail->>ServicioCliente: Notificacion enviada 

Este cambio de pensamiento de procedimientos a interacciones es quiza la tarea mas dificil para hacer la transicion al paradigma orientado a objetos. Pero no te desanimes! Estoy para ayudarte

## Metodos = mensajes

El paradigma de programacion orientada a objetos (como se definio originalmente por Alan Kay) solo tiene 2 conceptos: objetos y mensajes. Eso es todo lo que se necesita para _diseñar_ un modelo de objetos. Sin embargo en los lenguajes que soportan ese paradigma, se necesitan algunos conceptos adicionales. Estos son detalles de implementacion especificos al lenguaje, pero suelen confundirse con el paradigma en si. Uno de estos detalles es el concepto de metodo. Un metodo no es otra cosa que una pieza de codigo que es ejecutada en respuesta a un mensaje. Para simplificar la implementacion la mayoria de los lenguajes siguen una convencion sencilla: implementar una funcion publica en una clase. Tristemente esta implementacion ha llevado a muchos programadores a pensar que los mensajes y los metodos son los mismo. Para demostrarte que no es el caso, observa el siguiente codigo en C#:

In [21]:
interface IDoSomething
{
    void Message1();
    void Message2();
}

Como puedes ver, no hay codigo que se ejecute aqui. No hay _metodos_. Lo unico que encuentras son los mensajes. Una interface es solo una implementacion de la idea de contrato: una lista de mensajes que un objeto esta obligado a aceptar. Ahora mira:

In [22]:
class DoSomething: IDoSomething
{
    public void Message1()
    {
        Console.WriteLine("Message1");
    }

    public void Message2()
    {
        Console.WriteLine("Message2");
    }
}

Ahora si tenemos metodos: hay codigo que se ejecuta en respuesta a los mensajes definidos en la interface IDoSomething. Para efectos practicos podemos pensar en la _firma_ de la funcion como el _mensaje_ y en el _cuerpo_ como el _metodo_.

Esto es mas claro en lenguajes dinamicos. Observa el siguiente codigo en python:

In [ ]:
#!connect jupyter --kernel-name pythonkernel --kernel-spec python3

class Perro:
    def ladra(objeto):
        print(f'{objeto} está ladrando...')

pulgoso = Perro()
pulgoso.ladra()

Pon atencion en como el metodo `ladra` recibe un objeto, pero al enviar el mensaje `pulgoso.ladra()` no estamos pasando ningun argumento. El _runtime_ se encarga de eso. Lo que quiere decir que no _invocamos un metodo_ directamente. El runtime se encarga de ubicar el metodo asignado a ese mensaje y ejecutarlo con los argumentos necesarios. Eso pasa con muchos lenguajes pero de manera automatica. Normalmente el argumento se almacena en una variable llamada _this_ o _self_.

## Todos los objetos tienen clases

Otra de las cosas que causa interferencia en el pensamiento orientado a objetos es la idea de que un objeto debe tener un una clase. Esto nos limita al pensar que si no tiene una clase, no es un objeto. Un ejemplo claro de que esto no es cierto lo encontramos en lenguajes como self y javascript.

In [41]:
perro = {
    ladra: function() { console.log("barf barf") }
}

perro.ladra()

barf barf

Recordemos que un objeto es una cosa que responde a mensajes. En este caso 'perro' responde al mensaje 'ladra'. Asi que estamos frente a un objeto hecho y derecho.

Las clases, son _clases de objetos_ o sea que son un mecanismo para _clasificar_ a ciertos objetos que responden a un un grupo de mensajes: un contrato. Convenientemente, el codigo ejecutado en respuesta a esos eventos (metodos) tambien vive ahi. Pero esto no es necesario. Desafortunadamente muchos lenguajes definen clases como la unica forma de crear un objeto, lo que limita nuestra concepcion de lo que es un objeto (una instancia de una clase vs algo que responde mensajes), su creacion y la ubicacion de sus metodos.

## Todos los metodos viven en clases

Como ya demostramos, los objetos pueden existir sin necesidad de una clase. Pero entonces donde viven sus metodos?. Hay muchas opciones, por ejemplo un metodo puede vivir en el objeto mismo, esto es clasico en javascript:

In [25]:
function Perro() {
    return {
        ladra: function() { console.log("barf barf") }
    }
}

const bobby = Perro()
bobby.ladra()

barf barf

Tambien pueden vivir en el contexto donde se crea el objeto!

In [26]:
function Perro() {
    function ladrar() {
        console.log("barf barf")
    }

    return {
        ladra: ladrar
    }
}

const bobby = Perro()
bobby.ladra()

barf barf

A menudo, viven en otros objetos (delegacion):

In [27]:
function Perro() {
    return {
        ladra: function() { console.log("barf barf") }
    }
}

function Dobberman(perro) {
    return {
        ladra: perro.ladra
    }
}

const spike = new Dobberman(Perro())
spike.ladra()

barf barf

## Todos los objetos se crean con 'new'

Esta es otra confusion comun. Como estamos acostumbrados a definir clases para crear nuevos objetos, y como normalmente se usa la instruccion new para crearlo, entonces nos es dificil pensar en algo que no se inicialize con new como un objeto. Por ejemplo la siguiente expresion en C# crea un objeto:

In [28]:
var objeto1 = delegate(int x){ Console.WriteLine(x); };
objeto1.Invoke(42);
objeto1.Display()

42


System.Action`1[System.Int32] Target Submission#14+<>c Method Void <<Initialize>>b__0_0(Int32) Name <<Initialize>>b__0_0 DeclaringType Submission#14+<>c ReflectedType Submission#14+<>c MemberType Method MetadataToken 100663305 Module ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId e3557b8d-cb2a-400a-9374-03c927c19d2b MetadataToken 1 ScopeName ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14.dll Name <Unknown> Assembly ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null CodeBase file:///C:/Program Files/dotnet/shared/Microsoft.NETCore.App/9.0.1/System.Private.CoreLib.dll FullName ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null EntryPoint <null> DefinedTypes index value 0 Submission#14 1 Submission#14+<<Initialize>>d__0 2 Submission#14+<>c IsCollectible False ManifestModule ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14.dll MDStreamVersion 131072 FullyQualifiedName <Unknown> ModuleVersionId e3557b8d-cb2a-400a-9374-03c927c19d2b MetadataToken 1 ScopeName ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14.dll Name <Unknown> Assembly ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null CodeBase file:///C:/Program Files/dotnet/shared/Microsoft.NETCore.App/9.0.1/System.Private.CoreLib.dll FullName ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null EntryPoint <null> DefinedTypes [ Submission#14, Submission#14+< >d__0, Submission#14+<>c ] IsCollectible False ManifestModule ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14.dll ReflectionOnly False Location ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes [ Submission#14, Submission#14+< >d__0, Submission#14+<>c ] IsFullyTrusted True CustomAttributes [ [System.Runtime.CompilerServices.CompilationRelaxationsAttribute((Int32)8)], [System.Runtime.CompilerServices.RuntimeCompatibilityAttribute(WrapNonExceptionThrows = True)], [System.Diagnostics.DebuggableAttribute((System.Diagnostics.DebuggableAttribute+DebuggingModes)263)] ] EscapedCodeBase file:///C:/Program%20Files/dotnet/shared/Microsoft.NETCore.App/9.0.1/System.Private.CoreLib.dll Modules [ ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14.dll ] SecurityRuleSet None ModuleHandle System.ModuleHandle MDStreamVersion 131072 CustomAttributes index value 0 [System.Security.UnverifiableCodeAttribute()] 1 [System.Runtime.CompilerServices.RefSafetyRulesAttribute((Int32)11)] ReflectionOnly False Location ImageRuntimeVersion v4.0.30319 GlobalAssemblyCache False HostContext 0 IsDynamic False ExportedTypes index value 0 Submission#14 1 Submission#14+<<Initialize>>d__0 2 Submission#14+<>c IsFullyTrusted True CustomAttributes index value 0 [System.Runtime.CompilerServices.CompilationRelaxationsAttribute((Int32)8)] Constructor Void .ctor(Int32) ConstructorArguments [ (Int32)8 ] NamedArguments [ ] AttributeType System.Runtime.CompilerServices.CompilationRelaxationsAttribute 1 [System.Runtime.CompilerServices.RuntimeCompatibilityAttribute(WrapNonExceptionThrows = True)] Constructor Void .ctor() ConstructorArguments [ ] NamedArguments [ WrapNonExceptionThrows = True ] AttributeType System.Runtime.CompilerServices.RuntimeCompatibilityAttribute 2 [System.Diagnostics.DebuggableAttribute((System.Diagnostics.DebuggableAttribute+DebuggingModes)263)] Constructor Void .ctor(DebuggingModes) ConstructorArguments [ (System.Diagnostics.DebuggableAttribute+DebuggingModes)263 ] NamedArguments [ ] AttributeType System.Diagnostics.DebuggableAttribute EscapedCodeBase file:///C:/Program%20Files/dotnet/shared/Microsoft.NETCore.App/9.0.1/System.Private.CoreLib.dll Modules index value 0 ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14.dll MDStreamVersion 131072 FullyQualifiedName ModuleVersionId e3557b8d-cb2a-400a-9374-03c927c19d2b MetadataToken 1 ScopeName ℛ*9128019d-e8e7-4863-a424-3ac31f004cad#1-14

En este caso _objeto1_ se inicializa con la instruccion _delegate_ en lugar de _new_. Esta clase de objetos reciben el metodo en su constructor (todo lo que sigue despues de la palabra _delegate_) y lo ejecutan en respuesta a el mensaje _Invoke_. Como puedes ver tambien responden a otros mensajes como _Display_. Otro ejemplo seria: 

In [36]:
int[] unArreglo = [1,2,3];
Console.Write(unArreglo[2]);

3

Ves la instruccion _new_ en algun lado? esta forma de crear objetos se llama creacion literal. Es lo mismo con los numeros y los strings en C#.

In [42]:
var numero = 42;
Console.WriteLine(numero.ToString());

var saludo = "Hola Mundo";
Console.WriteLine(saludo.ToUpper());

42
HOLA MUNDO


Como puedes ver numero y saludo responden a mensajes (ToString y ToUpper en este caso), pero no vemos la instruccion _new_ en su creacion. 

## Resumen

Un modelo de objetos puede ser diseñado e implementado en cualquier lenguaje que soporte el paradigma orientado a objetos si nos enfocamos en los 2 conceptos esenciales del paradigma: objetos y mensajes.
Al definir un objeto como una cosa que acepta mensajes, nos liberamos de las diferentes formas de implementacion que varian de un lenguaje a otro. Estos detalles de implementacion imponen una restriccion artificial en nuestro pensamiento, ya que vienen de los lenguajes y no del paradigma en si. Asi que la proxima vez que trabajes en un lenguaje orientado a objetos, preguntate: es esto relevante para el diseño del modelo?